In [4]:
import sys
import wave
import math
import struct
import random
import argparse
from itertools import *

In [3]:
# def sine_wave(frequency=440.0, framerate=44100, amplitude=0.5):
#     if amplitude > 1.0: amplitude = 1.0
#     if amplitude < 0.0: amplitude = 0.0
#     return (float(amplitude) * math.sin(2.0*math.pi*float(frequency)*(float(i)/float(framerate))) for i in count(0))

In [5]:
def sine_wave(frequency=440.0, framerate=44100, amplitude=0.5):
    period = int(framerate / frequency)
    if amplitude > 1.0: amplitude = 1.0
    if amplitude < 0.0: amplitude = 0.0
    lookup_table = [float(amplitude) * math.sin(2.0*math.pi*float(frequency)*(float(i%period)/float(framerate))) for i in xrange(period)]
    return (lookup_table[i%period] for i in count(0))

In [6]:
def white_noise(amplitude=0.5):
    return (float(amplitude) * random.uniform(-1, 1) for _ in count(0))

In [7]:
noise = cycle(islice(white_noise(), 44100))

In [13]:
channels = ((sine_wave(200.0, amplitude=0.1), white_noise(amplitude=0.001)),
            (sine_wave(205.0, amplitude=0.1), white_noise(amplitude=0.001)))

In [14]:
def compute_samples(channels, nsamples=None):
    return islice(izip(*(imap(sum, izip(*channel)) for channel in channels)), nsamples)

In [20]:
nchannels=len(channels)
sampwidth = 2
framerate= 44100
nframes= 44100*10
w = wave.open('filename.wav', 'w')
w.setparams((nchannels, sampwidth, framerate, nframes, 'NONE', 'not compressed'))

In [23]:
max_amplitude = 32767.0
samples = [-32767, 32767]
samples = (int(sample * max_amplitude) for sample in samples)

In [24]:
struct.pack('h', 1000)

'\xe8\x03'

In [25]:
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return izip_longest(fillvalue=fillvalue, *args)

def write_wavefile(filename, samples, nframes=None, nchannels=2, sampwidth=2, framerate=44100, bufsize=2048):
    if nframes is None:
        nframes = -1

    w = wave.open(filename, 'w')
    w.setparams((nchannels, sampwidth, framerate, nframes, 'NONE', 'not compressed'))

    max_amplitude = float(int((2 ** (sampwidth * 8)) / 2) - 1)

    # split the samples into chunks (to reduce memory consumption and improve performance)
    for chunk in grouper(bufsize, samples):
        frames = ''.join(''.join(struct.pack('h', int(max_amplitude * sample)) for sample in channels) for channels in chunk if channels is not None)
        w.writeframesraw(frames)

    w.close()

    return filename